In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.sprint_lap_time`
UNION ALL
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.final_driver_positions_all`
"""
project_id = "perceptive-ivy-290216"
df_bq = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
GP=df_bq[((df_bq["GP"]=="Chinese Grand Prix") & (df_bq["Year"]==2024))]
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/4068188463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
5772,0 days 00:47:31.652000,ALB,23,NaT,1,1.0,NaT,NaT,NaT,0 days 00:00:32.807000,0 days 00:00:46.717000,NaT,0 days 00:46:44.999000,0 days 00:47:31.810000,280.0,269.0,255.0,311.0,False,MEDIUM,7.0,False,Williams,0 days 00:45:38.934000,2024-04-20 03:04:02.541,1,19.0,False,,False,False,2024,Chinese Grand Prix
4686,0 days 00:49:15.997000,ALB,23,0 days 00:01:44.345000,2,1.0,NaT,NaT,0 days 00:00:27.883000,0 days 00:00:31.831000,0 days 00:00:44.631000,0 days 00:47:59.526000,0 days 00:48:31.357000,0 days 00:49:15.988000,262.0,271.0,259.0,336.0,True,MEDIUM,8.0,False,Williams,0 days 00:47:31.652000,2024-04-20 03:05:55.259,1,19.0,False,,False,True,2024,Chinese Grand Prix
4687,0 days 00:51:00.045000,ALB,23,0 days 00:01:44.048000,3,1.0,NaT,NaT,0 days 00:00:27.426000,0 days 00:00:31.922000,0 days 00:00:44.700000,0 days 00:49:43.414000,0 days 00:50:15.336000,0 days 00:51:00.036000,270.0,266.0,262.0,332.0,True,MEDIUM,9.0,False,Williams,0 days 00:49:15.997000,2024-04-20 03:07:39.604,1,19.0,False,,False,True,2024,Chinese Grand Prix
4688,0 days 00:52:42.763000,ALB,23,0 days 00:01:42.718000,4,1.0,NaT,NaT,0 days 00:00:27.146000,0 days 00:00:31.390000,0 days 00:00:44.182000,0 days 00:51:27.182000,0 days 00:51:58.572000,0 days 00:52:42.754000,267.0,266.0,257.0,314.0,True,MEDIUM,10.0,False,Williams,0 days 00:51:00.045000,2024-04-20 03:09:23.652,1,19.0,False,,False,True,2024,Chinese Grand Prix
4689,0 days 00:54:25.291000,ALB,23,0 days 00:01:42.528000,5,1.0,NaT,NaT,0 days 00:00:27.216000,0 days 00:00:31.246000,0 days 00:00:44.066000,0 days 00:53:09.970000,0 days 00:53:41.216000,0 days 00:54:25.282000,271.0,268.0,258.0,313.0,True,MEDIUM,11.0,False,Williams,0 days 00:52:42.763000,2024-04-20 03:11:06.370,1,19.0,False,,False,True,2024,Chinese Grand Prix


In [4]:
# GP=GP[GP["Driver"]!="GAS"]

In [5]:
fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
#             markers=True,
        title="Sprint Race Position Ladder for the {} {}".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=1000, width=1200, template="plotly_dark",
             color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )

fig.update_layout(
    title_x=0.5,
    yaxis_range=[1, 21],
    margin=dict(l=10, r=5, t=55, b=20),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 1,
    dtick = 1
    ),

    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),

    margin_pad=10
)
fig.update_yaxes(autorange="reversed")

# fig.update_layout(
    # updatemenus=[
    #     {
    #         "buttons": [
    #             {
    #                 "label": t.name,
    #                 "method": "restyle",
    #                 "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
    #             }
    #             for t in fig.data
    #         ]
    #     }
    # ]
# )

In [25]:
fig.write_html("{} {} Sprint Race Ladder.html".format(year,gp))

In [ ]:
# Multi GP Plot

In [6]:
#2023
# race_multi=['Austrian Grand Prix','Azerbaijan Grand Prix','Belgian Grand Prix','Qatar Grand Prix','São Paulo Grand Prix','United States Grand Prix']
#2022
# race_multi=['Austrian Grand Prix','Emilia Romagna Grand Prix','São Paulo Grand Prix']
#2021
race_multi=['British Grand Prix','Italian Grand Prix','São Paulo Grand Prix']

year_multi=2021
for i in race_multi: 

    GP=df_bq[((df_bq["GP"]==i) & (df_bq["Year"]==year_multi))]
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP.head()
    fig=px.line(GP, x="LapNumber", y="Position", color='Driver',
            title="Race Position Ladder for the {} {}".format(year_multi,i),
            hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
            height=1000, width=1200, template="plotly_dark",
                color_discrete_map={
                 "ALO": "#0090FF",
                 "BOT": "#00D2BE",
                 "GAS": "#2B4562",
                 "GIO": "#900000",               
                 "HAM": "#00D2BE",
                 "KUB": "#005AFF",
                 "LAT": "#005AFF",
                 "LEC": "#DC0004",
                 "MAZ": "#b6babd",
                 "MSC": "#b6babd",
                 "NOR": "#FF9800",
                 "OCO": "#0090FF",
                 "PER": "#0600EF",
                 "RAI": "#900000",
                 "RIC": "#FF9800",
                 "RUS": "#005AFF",
                 "SAI": "#DC0004",
                 "STR": "#006F62",
                 "TSU": "#2B4562",
                 "VER": "#0600EF",
                 "VET": "#006F62"
            },
            )

    fig.update_layout(
        title_x=0.5,
        yaxis_range=[1, 21],
        margin=dict(l=10, r=5, t=55, b=20),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="White"
        ),

        yaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
        ),

        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),

        margin_pad=10
    )
    fig.update_yaxes(autorange="reversed")
    fig.write_html("{} {}Sprint Race Ladder.html".format(year_multi,i))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/3986341913.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/3986341913.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/3986341913.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [7]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
GP=GP[GP["LapTime (s)"]<500]

In [7]:
fig_all_laps=px.scatter(GP, 
        x="LapNumber", 
        y="LapTime (s)", 
        color='Driver',
        title="All Lap Times for the {} {} Sprint Race".format(year,gp),
        hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
        height=700,
          width=1200, 
        template="plotly_dark",
        color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          },
        )
fig_all_laps.update_layout(
    title_x=0.5,
    hoverlabel=dict(
    bgcolor="white",
    font_size=16,
    font_family="PT Sans Narrow",
    # margin=dict(l=5, r=5, t=35, b=5),
    ),
    updatemenus=[
        {
            "buttons": [
                {
                    "label": t.name,
                    "method": "restyle",
                    "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                }
                for t in fig.data
            ]
        }
    ],
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),

    yaxis = dict(
    tickmode = 'linear',
    tick0 = 0,
    dtick = 2
    ),

    margin_pad=10
)

In [8]:
fig_all_laps.write_html("{} {} Sprint Race All Lap Times.html".format(year,gp))

In [ ]:
#Multi GP Plot

In [11]:
#2023
# race_multi=['Austrian Grand Prix','Azerbaijan Grand Prix','Belgian Grand Prix','Qatar Grand Prix','São Paulo Grand Prix','United States Grand Prix']
#2022
# race_multi=['Austrian Grand Prix','Emilia Romagna Grand Prix','São Paulo Grand Prix']
#2021
race_multi=['British Grand Prix','Italian Grand Prix','São Paulo Grand Prix']

year_multi=2021
for i in race_multi:
    GP=df_bq[((df_bq["GP"]==i) & (df_bq["Year"]==year_multi))]
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP=GP[GP["LapTime (s)"]<500]
    fig_all_laps=px.scatter(GP, 
            x="LapNumber", 
            y="LapTime (s)", 
            color='Driver',
            title="All Lap Times for the {} {}".format(year_multi,i),
            hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
            height=700,
            width=1200, 
            template="plotly_dark",
            color_discrete_map={
                 "ALO": "#0090FF",
                 "BOT": "#00D2BE",
                 "GAS": "#2B4562",
                 "GIO": "#900000",               
                 "HAM": "#00D2BE",
                 "KUB": "#005AFF",
                 "LAT": "#005AFF",
                 "LEC": "#DC0004",
                 "MAZ": "#b6babd",
                 "MSC": "#b6babd",
                 "NOR": "#FF9800",
                 "OCO": "#0090FF",
                 "PER": "#0600EF",
                 "RAI": "#900000",
                 "RIC": "#FF9800",
                 "RUS": "#005AFF",
                 "SAI": "#DC0004",
                 "STR": "#006F62",
                 "TSU": "#2B4562",
                 "VER": "#0600EF",
                 "VET": "#006F62"
            },
            )
    fig_all_laps.update_layout(
        title_x=0.5,
        hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow",
        ),
        updatemenus=[
            {
                "buttons": [
                    {
                        "label": t.name,
                        "method": "restyle",
                        "args": [{"visible": [t2.name == t.name for t2 in fig.data]}],
                    }
                    for t in fig.data
                ]
            }
        ],
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="White"
        ),

        yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 2
        ),

        margin_pad=10
    )
    fig_all_laps.write_html("{} {} Sprint Race All Lap Times.html".format(year_multi,i))

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/2594178583.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/2594178583.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_9305/2594178583.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [29]:
# GP=GP.sort_values(['LapNumber', "Driver"])
# GP.tail()

In [30]:
# fig2=px.scatter(GP,
#         x="LapNumber",
#         y="Position",
#         color='DriverNumber',
#         animation_frame="LapNumber",
#         title="Race Position Ladder for the {} {}".format(year,gp),
#         # animation_group='LapNumber',
#         range_x=[0,len(GP['LapNumber'].unique())+1],
#         text='Driver',
#         range_y=[0,20],
#         hover_data=['Compound','TyreLife','Team','LapTime','Stint', 'PitInTime', 'PitOutTime', 'Sector1Time', 'Sector2Time', 'LapStartTime', 'Year'],
#         height=1000, width=1800, template="plotly_dark",
#            color_discrete_map={
#                  "ALB": "#64C4FF",
#                  "ALO": "#229971",
#                  "BOT": "#52e252",
#                  "GAS": "#0093cc",
#                  "HAM": "#27F4D2",
#                  "HUL": "#B6BABD",
#                  "LEC": "#E80020",
#                  "MAG": "#B6BABD",
#                  "NOR": "#FF8000",
#                  "OCO": "#0093cc",
#                  "PER": "#3671C6",
#                  "PIA": "#FF8000",
#                  "RIC": "#6692FF",
#                  "RUS": "#27F4D2",
#                  "SAI": "#E80020",
#                  "SAR": "#37BEDD",
#                  "STR": "#229971",
#                  "TSU": "#6692FF",
#                  "VER": "#3671C6",
#                  "ZHO": "#52e252"
#           },
#          )


# fig2.update_yaxes(autorange="reversed")

# fig.update_traces(textposition='middle right')

# fig2.update_layout(
#     title_x=0.5,
#     hoverlabel=dict(
#     bgcolor="white",
#     font_size=16,
#     font_family="PT Sans Narrow"
#     ),
#     font=dict(
#         family="PT Sans Narrow",
#         size=14,
#         color="White"
#     ),

#     yaxis = dict(
#     tickmode = 'linear',
#     tick0 = 0,
#     dtick = 1
#     ),

#     margin_pad=10
# )

# fig2.update(layout_coloraxis_showscale=False)
# # for x,y in zip(GP[GP["Driver"]=="SAI"].Driver, GP[GP["Driver"]=="SAI"].Position):
# #   for png in (Path.cwd().joinpath("f1").glob("*.png")):
# #     if str.split(str.split(str(png),".")[0],"/")[3]==x:
# #       image=str(png)
# #       fig2.add_layout_image(
# #           x=x,
# #           y=y+4,
# #           source=Image.open(image),
# #           xref="x",
# #           yref="y",
# #           sizex=5,
# #           sizey=5,
# #           xanchor="center",
# #           yanchor="middle",
# #       )
# fig2.show()

In [31]:
# fig2.write_html("{} {} Race Animation.html".format(year,gp))

# Fastest Driver in each lap

In [86]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["RK"] = GP_Rk.groupby("LapNumber")["LapTime"].rank(method="dense", ascending=True)
GP_FASTEST_LAPS=GP_Rk[GP_Rk["RK"]==1.0].sort_values("LapNumber")
GP_FASTEST_LAPS=GP_FASTEST_LAPS[["Time","Driver","Team","LapTime","LapNumber","Stint","PitOutTime","PitInTime","Sector1Time","Sector2Time","Sector3Time","SpeedI1","SpeedI2","TyreLife","TrackStatus","Position","Year","GP","RK"]]
GP_FASTEST_LAPS.head()

,Time,Driver,Team,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,TyreLife,TrackStatus,Position,Year,GP,RK
543798,0 days 00:58:40.015000,RUS,Mercedes,00:01:43.723000,1,1.0,NaT,NaT,NaT,0 days 00:00:31.645000,0 days 00:00:37.671000,190.0,264.0,1.0,12,1.0,2024,Canadian Grand Prix,1.0
108881,0 days 01:00:16.571000,RUS,Mercedes,00:01:36.556000,2,1.0,NaT,NaT,0 days 00:00:28.287000,0 days 00:00:30.597000,0 days 00:00:37.672000,198.0,264.0,2.0,1,1.0,2024,Canadian Grand Prix,1.0
74979,0 days 01:02:01.046000,MAG,Haas F1 Team,00:01:36.134000,3,1.0,NaT,NaT,0 days 00:00:27.936000,0 days 00:00:31.214000,0 days 00:00:36.984000,198.0,268.0,3.0,1,4.0,2024,Canadian Grand Prix,1.0
364215,0 days 01:03:35.039000,MAG,Haas F1 Team,00:01:33.993000,4,1.0,NaT,NaT,0 days 00:00:27.334000,0 days 00:00:29.992000,0 days 00:00:36.667000,215.0,265.0,4.0,1,4.0,2024,Canadian Grand Prix,1.0
129219,0 days 01:05:04.364000,VER,Red Bull Racing,00:01:33.859000,5,1.0,NaT,NaT,0 days 00:00:27.574000,0 days 00:00:30.293000,0 days 00:00:35.992000,200.0,267.0,5.0,1,2.0,2024,Canadian Grand Prix,1.0
